In [ ]:
import ultralytics
from ultralytics import YOLO
import argparse
import pytesseract
import cv2
import easyocr

In [ ]:
# This is the argument parser to help me detect what the users what the system to perform


def arg_parser():
    parse_input = argparse.ArgumentParser(
        description="This project is for detecting cars number plate"
    )
    # parse_input.add_argument(
    #     "--mode",
    #     type=str,
    #     # required=True,
    #     help="Please select test or train",
    #     choices=["train", "test"],
    # )
    # parse_input.add_argument("--image", type=str, help="Image")
    # parse_input.add_argument("--video", help="Image file only", type=str, )
    # parse_input.add_argument("epoch", type=int,)

    return parse_input.parse_args([])

In [ ]:
# models and training


def test_trainer(args):
    model = YOLO("yolov8n.pt")
    model.train(data=args.data, epochs=args.epochs, imgsz=args.images)

In [ ]:
if __name__ == "__main__":
    args = arg_parser()
    args.mode = 'train'
    args.epochs = 10
    args.data = 'datasets/data.yaml'
    args.images = 640
    if args.mode == 'train':
        test_trainer(args)
        

In [ ]:
# image model

saved_model = YOLO("runs/detect/train15/weights/best.pt")  # or last.pt
saved_model.predict(source="screenshot3.png", save=True)

# result = cv2.imread('debug_plate.jpg')
result = cv2.imread('runs/detect/predict7/screenshot3.jpg')

for box in r.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped = result[y1:y2, x1:x2]

    # Resize for better OCR
    cropped = cv2.resize(cropped, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    cv2.imwrite("debug_plate.jpg", cropped)


    # Grayscale and preprocess
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    denoised = cv2.bilateralFilter(gray,  d=11, sigmaColor=17, sigmaSpace=17)
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite("1_gray.jpg", gray)
    cv2.imwrite("2_denoised.jpg", denoised)
    cv2.imwrite("3_thresh.jpg", thresh)
    cv2.imwrite("4_morphed.jpg", morphed)
    
    # OCR
    reader = easyocr.Reader(['en'])
    rz = reader.readtext(thresh)

    # Step 3: Loop over results
    for bbox, text, conf in rz:
        print("🔠 Detected Plate Text:", text)

In [23]:
from ultralytics import YOLO
import cv2
import easyocr

model = YOLO("runs/detect/train15/weights/best.pt")
reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture("video5.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter("output_annotated_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    resized = cv2.resize(frame, (640, 640))
    results = model.predict(source=resized, conf=0.25, save=False)

    x_ratio = frame.shape[1] / 640
    y_ratio = frame.shape[0] / 640

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Rescale to original frame size
            x1 = int(x1 * x_ratio)
            x2 = int(x2 * x_ratio)
            y1 = int(y1 * y_ratio)
            y2 = int(y2 * y_ratio)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            cropped = frame[y1:y2, x1:x2]
            if cropped.size == 0:
                continue  # Skip empty crops

            cropped = cv2.resize(cropped, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

            gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
            denoised = cv2.bilateralFilter(gray, 11, 17, 17)
            thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                           cv2.THRESH_BINARY, 11, 2)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
            morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

            ocr_results = reader.readtext(morphed)
            for _, text, conf in ocr_results:
                if conf > 0.5:
                    plate_text = text.strip()
                    cv2.putText(frame, plate_text, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Annotated video saved.")



0: 640x640 (no detections), 165.5ms
Speed: 5.0ms preprocess, 165.5ms inference, 7.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 88.5ms
Speed: 1.9ms preprocess, 88.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 90.4ms
Speed: 2.0ms preprocess, 90.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 76.5ms
Speed: 1.6ms preprocess, 76.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.2ms
Speed: 1.6ms preprocess, 82.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 64.9ms
Speed: 1.5ms preprocess, 64.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 72.7ms
Speed: 1.1ms preprocess, 72.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 72.6ms
Speed: 1.2ms preprocess, 72.6ms